## 1. Configuração Banco de Dados ##

###  Conexão com o Banco de Dados ###

In [1]:
#pip install mysql-connector-python
import mysql.connector
from sqlalchemy import create_engine, text

In [ ]:
USER = "your_user"
PASSWORD = "yout_password"
HOST = "localhost"
DATABASE = "your_database" 

In [21]:
# Criar conexão usando SQLAlchemy
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DATABASE}")

### Criação das Tabelas ###

In [22]:
with engine.connect() as connection:
    tables_to_drop = ["Metrics", "Experiments", "LearnStrategies", "Hyperparameters", "Models"]
    for table in tables_to_drop:
        connection.execute(text(f"DROP TABLE IF EXISTS {table};"))
    connection.commit() # Confirma a remoção das tabelas

In [23]:
tables = {
    "Models": """
    CREATE TABLE IF NOT EXISTS Models (
        id INT AUTO_INCREMENT PRIMARY KEY,
        type INT,  -- 0: Classificação, 1: Regressão
        algorithm VARCHAR(50)
    )
    """,
    
    "Hyperparameters": """
    CREATE TABLE IF NOT EXISTS Hyperparameters (
        id INT AUTO_INCREMENT PRIMARY KEY,
        model_id INT,
        name VARCHAR(50),
        value VARCHAR(50),
        FOREIGN KEY (model_id) REFERENCES Models(id) ON DELETE CASCADE
    )
    """,
    
    "LearnStrategies": """
    CREATE TABLE IF NOT EXISTS LearnStrategies (
        id INT AUTO_INCREMENT PRIMARY KEY,
        model_id INT NOT NULL,
        preprocessing_type VARCHAR(50) NOT NULL,  -- Scaling, Sampling, Feature Selection
        data_sampling VARCHAR(50) NOT NULL,  -- Undersampling, Oversampling, Stratified Sampling
        type VARCHAR(50) NOT NULL,  -- Cross-Validation, Hold-out
        len_data JSON NOT NULL,  -- Lista [Treino, Teste, Validação]
        FOREIGN KEY (model_id) REFERENCES Models(id) ON DELETE CASCADE
    )
    """,
    
    "Experiments": """
    CREATE TABLE IF NOT EXISTS Experiments (
        id INT AUTO_INCREMENT PRIMARY KEY,
        model_id INT,
        dataset VARCHAR(50),
        date_created TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (model_id) REFERENCES Models(id) ON DELETE CASCADE
    )
    """,

    "Metrics": """
    CREATE TABLE IF NOT EXISTS Metrics (
        id INT AUTO_INCREMENT PRIMARY KEY,
        experiment_id INT,
        type VARCHAR(50),
        value FLOAT,
        FOREIGN KEY (experiment_id) REFERENCES Experiments(id) ON DELETE CASCADE
    )
    """
}

In [24]:
with engine.connect() as connection:
    for table_name, sql_query in tables.items():
        connection.execute(text(sql_query)) # Use text() here!
        print(f"Tabela {table_name} criada com sucesso! ✅")

print("Todas as tabelas foram criadas corretamente! 🎉")

Tabela Models criada com sucesso! ✅
Tabela Hyperparameters criada com sucesso! ✅
Tabela LearnStrategies criada com sucesso! ✅
Tabela Experiments criada com sucesso! ✅
Tabela Metrics criada com sucesso! ✅
Todas as tabelas foram criadas corretamente! 🎉


## 2.Funções de Treinamento e Avaliação ##

### Importação das Bibliotecas ###

In [25]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt #pip install matplotlib
import seaborn as sns #pip install seaborn

In [26]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

### Carregar Datasets ###

In [27]:
def load_dataset(name):
    from sklearn.datasets import load_iris, load_breast_cancer, load_diabetes, fetch_california_housing
    
    if name == "iris":
        data = load_iris()
    elif name == "breast_cancer":
        data = load_breast_cancer()
    elif name == "boston":
        from sklearn.datasets import load_diabetes  # Boston foi removido, substituído por Diabetes
        data = load_diabetes()
    elif name == "california":
        data = fetch_california_housing()
    else:
        raise ValueError("Dataset não suportado")
    
    return pd.DataFrame(data.data, columns=data.feature_names), pd.Series(data.target)


In [28]:
def train_and_save_model(dataset_name, model_type, algorithm, model, params={}):
    # Carregar dataset
    X, y = load_dataset(dataset_name)
    
    # Normalizar os dados
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Separar treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Treinar o modelo
    model.set_params(**params)
    model.fit(X_train, y_train)

    # Fazer previsões
    y_pred = model.predict(X_test)
    
    if model_type == "classification":
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, average='weighted'),
            "recall": recall_score(y_test, y_pred, average='weighted'),
            "f1_score": f1_score(y_test, y_pred, average='weighted'),
            "auc_roc": roc_auc_score(y_test, model.decision_function(X_test), multi_class='ovr') if hasattr(model, "decision_function") else None
        }
    else:  # Regression
        metrics = {
            "mse": mean_squared_error(y_test, y_pred),
            "rmse": np.sqrt(mean_squared_error(y_test, y_pred)),
            "mae": mean_absolute_error(y_test, y_pred),
            "r2": r2_score(y_test, y_pred)
        }

    with engine.connect() as connection:
        try:
            # Save model
            result = connection.execute(text("INSERT INTO Models (type, algorithm) VALUES (:type, :algorithm)"),
                                        {"type": 0 if model_type == "classification" else 1, "algorithm": algorithm})
            model_id = result.lastrowid

            # Save hyperparameters
            for name, value in params.items():
                connection.execute(text("INSERT INTO Hyperparameters (model_id, name, value) VALUES (:model_id, :name, :value)"),
                                 {"model_id": model_id, "name": name, "value": str(value)})

            # Save experiment
            result = connection.execute(text("INSERT INTO Experiments (model_id, dataset) VALUES (:model_id, :dataset)"),
                                        {"model_id": model_id, "dataset": dataset_name})
            experiment_id = result.lastrowid


            # Now save the metrics (metrics is defined now!)
            for metric, value in metrics.items():
                connection.execute(text("INSERT INTO Metrics (experiment_id, type, value) VALUES (:experiment_id, :type, :value)"),
                             {"experiment_id": experiment_id, "type": metric, "value": value})

            connection.commit()
            print(f"Modelo {algorithm} treinado e salvo com sucesso!")

        except Exception as e:
            connection.rollback()
            print(f"Erro ao salvar o modelo: {e}")
            raise

In [29]:
# Treinar e armazenar modelos
train_and_save_model("iris", "classification", "RandomForest", RandomForestClassifier(), {"n_estimators": 100})
train_and_save_model("breast_cancer", "classification", "SVM", SVC(probability=True), {"kernel": "linear"})
train_and_save_model("boston", "regression", "DecisionTree", DecisionTreeRegressor(), {"max_depth": 5})
train_and_save_model("california", "regression", "XGBoost", XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, subsample=0.7, colsample_bytree=0.7), {"n_estimators": 100})

Modelo RandomForest treinado e salvo com sucesso!
Modelo SVM treinado e salvo com sucesso!
Modelo DecisionTree treinado e salvo com sucesso!
Modelo XGBoost treinado e salvo com sucesso!


In [30]:
def fetch_results():
    with engine.connect() as connection:  # Use SQLAlchemy connection
        df = pd.read_sql(text("""
            SELECT e.dataset, m.algorithm, me.type, me.value 
            FROM Experiments e 
            JOIN Models m ON e.model_id = m.id
            JOIN Metrics me ON e.id = me.experiment_id
        """), connection) # Pass the SQLAlchemy connection to read_sql

        df_agg = df.groupby(['dataset', 'algorithm', 'type'])['value'].mean().reset_index() 


        return df_agg.pivot(index=['dataset', 'algorithm'], columns='type', values='value')


df_results = fetch_results()
df_results


,type,accuracy,auc_roc,f1_score,mae,mse,precision,r2,recall,rmse
dataset,algorithm,,,,,,,,,
boston,DecisionTree,NaN,NaN,NaN,47.572000,3698.810000,NaN,0.301868,NaN,60.817800
breast_cancer,SVM,0.95614,0.996397,0.956237,NaN,NaN,0.956488,NaN,0.95614,NaN
california,XGBoost,NaN,NaN,NaN,0.329314,0.239564,NaN,0.817184,NaN,0.489453
iris,RandomForest,1.00000,NaN,1.000000,NaN,NaN,1.000000,NaN,1.00000,NaN


In [31]:
def fetch_detailed_results():
    with engine.connect() as connection:  # Use SQLAlchemy connection
        df = pd.read_sql(text("""
            SELECT e.dataset, m.algorithm, m.type AS model_type, me.type AS metric_type, me.value
            FROM Experiments e
            JOIN Models m ON e.model_id = m.id
            JOIN Metrics me ON e.id = me.experiment_id
        """), connection)  # Pass SQLAlchemy connection
        
        df_pivot = df.pivot_table(index=['dataset', 'algorithm', 'model_type'], columns='metric_type', values='value', aggfunc='mean').reset_index()
        return df_pivot

df_detailed_results = fetch_detailed_results()

df_detailed_results['model_type'] = df_detailed_results['model_type'].map({0: 'Classificação', 1: 'Regressão'})

classification_results = df_detailed_results[df_detailed_results['model_type'] == 'Classificação'].dropna(axis=1, how='all')
regression_results = df_detailed_results[df_detailed_results['model_type'] == 'Regressão'].dropna(axis=1, how='all')

In [32]:
classification_results


metric_type,dataset,algorithm,model_type,accuracy,auc_roc,f1_score,precision,recall
1,breast_cancer,SVM,Classificação,0.95614,0.996397,0.956237,0.956488,0.95614
3,iris,RandomForest,Classificação,1.00000,NaN,1.000000,1.000000,1.00000


In [33]:
regression_results

metric_type,dataset,algorithm,model_type,mae,mse,r2,rmse
0,boston,DecisionTree,Regressão,47.572000,3698.810000,0.301868,60.817800
2,california,XGBoost,Regressão,0.329314,0.239564,0.817184,0.489453


In [34]:
train_and_save_model("iris", "classification", "DecisionTree", DecisionTreeClassifier(criterion="entropy", max_depth=3), {"criterion": "entropy", "max_depth": 3})

Modelo DecisionTree treinado e salvo com sucesso!


In [35]:
def fetch_detailed_results():
    with engine.connect() as connection:  # Usar conexão SQLAlchemy
        df = pd.read_sql(text("""
            SELECT e.dataset, m.algorithm, m.type AS model_type, me.type AS metric_type, me.value
            FROM Experiments e
            JOIN Models m ON e.model_id = m.id
            JOIN Metrics me ON e.id = me.experiment_id
        """), connection)  # Passar a conexão para o Pandas

        df_pivot = df.pivot_table(index=['dataset', 'algorithm', 'model_type'], columns='metric_type', values='value', aggfunc='mean').reset_index()
        return df_pivot

In [38]:
df_detailed_results02 = fetch_detailed_results()

df_detailed_results02['model_type'] = df_detailed_results02['model_type'].map({0: 'Classificação', 1: 'Regressão'})

classification_results_02 = df_detailed_results02[df_detailed_results02['model_type'] == 'Classificação'].dropna(axis=1, how='all')
regression_results_02 = df_detailed_results02[df_detailed_results02['model_type'] == 'Regressão'].dropna(axis=1, how='all')

In [40]:
df_detailed_results02

metric_type,dataset,algorithm,model_type,accuracy,auc_roc,f1_score,mae,mse,precision,r2,recall,rmse
0,boston,DecisionTree,Regressão,NaN,NaN,NaN,47.572000,3698.810000,NaN,0.301868,NaN,60.817800
1,breast_cancer,SVM,Classificação,0.95614,0.996397,0.956237,NaN,NaN,0.956488,NaN,0.95614,NaN
2,california,XGBoost,Regressão,NaN,NaN,NaN,0.329314,0.239564,NaN,0.817184,NaN,0.489453
3,iris,DecisionTree,Classificação,1.00000,NaN,1.000000,NaN,NaN,1.000000,NaN,1.00000,NaN
4,iris,RandomForest,Classificação,1.00000,NaN,1.000000,NaN,NaN,1.000000,NaN,1.00000,NaN


In [41]:
classification_results_02

metric_type,dataset,algorithm,model_type,accuracy,auc_roc,f1_score,precision,recall
1,breast_cancer,SVM,Classificação,0.95614,0.996397,0.956237,0.956488,0.95614
3,iris,DecisionTree,Classificação,1.00000,NaN,1.000000,1.000000,1.00000
4,iris,RandomForest,Classificação,1.00000,NaN,1.000000,1.000000,1.00000


In [42]:
regression_results_02

metric_type,dataset,algorithm,model_type,mae,mse,r2,rmse
0,boston,DecisionTree,Regressão,47.572000,3698.810000,0.301868,60.817800
2,california,XGBoost,Regressão,0.329314,0.239564,0.817184,0.489453


In [44]:
import pandas as pd
from sqlalchemy import create_engine, text

# ... (your existing connection details USER, PASSWORD, HOST, DATABASE)

engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DATABASE}")

def fetch_models_and_hyperparams():
    with engine.connect() as connection:  # Use SQLAlchemy connection
        df = pd.read_sql(text("""
            SELECT e.dataset, m.algorithm, h.name AS hyperparameter_name, h.value AS hyperparameter_value
            FROM Experiments e
            JOIN Models m ON e.model_id = m.id
            JOIN Hyperparameters h ON m.id = h.model_id
        """), connection)  # Pass SQLAlchemy connection

        results = {}
        for (dataset, algorithm), group in df.groupby(['dataset', 'algorithm']):
            hyperparams = "; ".join(f"{row['hyperparameter_name']} = {row['hyperparameter_value']}" for _, row in group.iterrows())
            results[f"{dataset} ({algorithm})"] = f"{{ {hyperparams} }}"

        return results

models_and_hyperparams = fetch_models_and_hyperparams()
for model, hyperparams in models_and_hyperparams.items():
    print(f"{model} : {hyperparams}")


boston (DecisionTree) : { max_depth = 5 }
breast_cancer (SVM) : { kernel = linear }
california (XGBoost) : { n_estimators = 100 }
iris (DecisionTree) : { criterion = entropy; max_depth = 3 }
iris (RandomForest) : { n_estimators = 100 }
